In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Conv2D, Concatenate, MaxPool2D, Reshape, Flatten, Dropout
from keras.layers import ReLU
from keras.layers import BatchNormalization, LSTM, Bidirectional

from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.layers import Add

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
from keras import backend as K

In [4]:
# !python3 -m pip install gensim

In [5]:
from gensim.models import Word2Vec

In [6]:
%matplotlib inline

In [7]:
# !python3 -m pip install -U nltk
# !python3 -m nltk.downloader punkt

In [8]:
import nltk

In [9]:
from nltk.corpus import stopwords

In [10]:
path = 'epi_r.csv'
# path = 'recipes_with_calories.csv'

In [11]:
df = pd.read_csv(path)
# df = pd.read_csv(path, sep="|", index_col=0)

In [12]:
all_columns = list(df.columns)
len(all_columns)

680

In [13]:
df.dropna(axis=0).shape, df.shape

((15864, 680), (20052, 680))

In [14]:
for column in df.columns:
    if df[column].isnull().any():
        print(column)
#         df[column] = df[column].fillna(df[column].mean())

calories
protein
fat
sodium


In [15]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def get_small_dict(df, keep_columns, index):
    my_dict = df.iloc[index][keep_columns].to_dict()
    my_dict['tags'] = [
        key for key, value in df.iloc[index].to_dict().items() if value != 0 and key not in keep_columns]
    my_dict['title'] = my_dict['title'].strip()
    my_dict['tags'] = [
        str(unicodedata.normalize('NFKD', t).encode('ascii','ignore'), encoding='ascii') for t in my_dict['tags']
    ]
    return my_dict

In [17]:
keep_columns = ['title', 'rating', 'calories', 'protein', 'fat','sodium']

In [18]:
def remove_outliers_with_tukey(df):
    print('There were {} samples before outlier removal.'.format(len(df)))

    # Remove calories outliers by the Tukey rule
    Q3 = np.percentile(df[regr_column], 75)
    Q1 = np.percentile(df[regr_column], 25)
    IQR = Q3 - Q1

    outliers = df[regr_column].loc[
        (df[regr_column] < (Q1 - 1.5 * IQR)) | 
        (df[regr_column] > (Q3 + 1.5 * IQR))
    ].index
    
    df = df.drop(outliers)
    
    print('There are now {} samples.'.format(len(cut_df)))

    return df

In [20]:
# pick regression column
regr_column = 'calories'
cut_df = df[df[regr_column] == df[regr_column]]

cut_df = remove_outliers_with_tukey(df=cut_df)

# transform df in list of dictionaries for processing
regr_data = []
for i in range(len(cut_df)):
    regr_data.append(get_small_dict(cut_df, keep_columns, i))

There were 15935 samples before outlier removal.
There are now 15935 samples.


In [21]:
def remove_punctuation_signs(regr_data):
    for d in regr_data:
        if 'tags' not in d:
            raise ValueError('Expected to have <tags> key in each of the regression input data dictionary!')
    corpus = ['|'.join(
        d['tags'] + 
        [w.lower() for w in nltk.word_tokenize(d['title']) if
             w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
    ) for d in regr_data]
    return corpus

In [22]:
# make sure the data doesn't have any punctuation signs
corpus = remove_punctuation_signs(regr_data)

In [23]:
# pick tokenization method
def tokenize(word_list):
    return word_list.split('|')

In [24]:
def get_y_labels(regr_column, regr_data):
    return [d[regr_column] for d in regr_data]

In [25]:
y_label = get_y_labels(regr_column, regr_data)

In [26]:
if len(corpus) != len(y_label):
    print('[ERROR] There is a mismatch between the size of the input data ' +
          ' ({} elements) and the size of labels ({} elements)!'.format(len(corpus), len(y_label)))

## Try a vanilla regression (CountVectorizer + LinearRegression from sklearn)

In [28]:
# fit vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize)
X = vectorizer.fit_transform(corpus)

# train, test data
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y_label, test_size=0.2, random_state=42)

# sklearn LinearRegression model
regr_sklearn = LinearRegression()
regr_sklearn.fit(X_train, y_train)

# predict data
y_pred = regr_sklearn.predict(X_test)

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  6203187741909.541
Std:  252.29854879882052
Mean:  373.96012064343165
353.99687688163283 874.0
1009143909089.2003 772.0
646.5372827068254 591.0
2135712259944.3208 557.0
576.8568622331953 582.0
829.1368304949141 811.0
250.67473332694533 292.0
326.827733204875 206.0
860797942518.8972 954.0
236.38335149100783 234.0


## Try Word2Vec embeddings + Keras NN

### Word2Vec embedding

In [29]:
vocab = set()
for word_list in corpus:
    vocab |= set(tokenize(word_list))

print("Choose embedding size between {} and {}".format(np.log2(len(vocab)), np.power(len(vocab), 1/4) * 10))

w2v_corpus = []
for word_list in corpus:
    w2v_corpus.append(tokenize(word_list))

Choose embedding size between 12.533816389433365 and 87.75334365864005


In [30]:
EMBEDDING_SIZE = 50

In [31]:
w2v_model = Word2Vec(w2v_corpus, size=EMBEDDING_SIZE, window=5, min_count=1, workers=-1)

print("Embedding size is {}".format(w2v_model.wv.vector_size))

Embedding size is 50


In [32]:
sequence_lengths = [len(w) for w in w2v_corpus]

print("Sequence length should be {} ".format(pd.Series(sequence_lengths).describe()['mean']))

Sequence length should be 16.97908566832015 


In [33]:
FIXED_LENGTH = 17

In [34]:
def generate_embeddings_matrix(sample, model, fixed_length=17):
    embedding_size = model.wv.vector_size
    embeddings = []
    for sen in sample:
        for word in sen:
            if word not in model.wv.vocab:
                continue
            embeddings.append(model.wv[word])
            if len(embeddings) >= fixed_length:
                break

    embeddings = np.array(embeddings)

    while len(embeddings) < fixed_length:
        padding_length = fixed_length - len(embeddings)
        embeddings = np.concatenate((embeddings, embeddings[:padding_length]))
    if len(embeddings) > fixed_length:
        embeddings = embeddings[:fixed_length]

    return embeddings

In [35]:
def generate_input_matrix(sample, fixed_length=17):
    
    inputs = np.array(sample)

    while len(inputs) < fixed_length:
        padding_length = fixed_length - len(inputs)
        inputs = np.concatenate((inputs, inputs[:padding_length]))
    if len(inputs) > fixed_length:
        inputs = inputs[:fixed_length]

    return inputs

In [36]:
# Generate list of input data (embedding matrix) and make sure every input has FIXED_LENGTH elements
X = []
for sample in w2v_corpus:
    embedded_sample = generate_embeddings_matrix(sample, w2v_model, FIXED_LENGTH)
    if embedded_sample.shape != (FIXED_LENGTH, EMBEDDING_SIZE):
        print(embedded_sample.shape)
    X.append(embedded_sample)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

### Keras NN

In [38]:
FIXED_LENGTH = 17
EMBEDDING_SIZE = 50

BATCH_SIZE = 16
EPOCHS=200

In [39]:
def baseline_model():
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH, EMBEDDING_SIZE), name='input')
    flat = Flatten()(input_layer)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(flat)
    output = Dense(1, kernel_initializer='normal')(z)
    model = Model(inputs=[input_layer], outputs=[output])
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [51]:
model = baseline_model()

In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 17, 50)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 850)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 32)                27232     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 27,265
Trainable params: 27,265
Non-trainable params: 0
_________________________________________________________________


In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

In [54]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [56]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/200
11934/11934 [==============================] - 2s 132us/step - loss: 192319.3075
Epoch 2/200
11934/11934 [==============================] - 1s 103us/step - loss: 125901.1676
Epoch 3/200
11934/11934 [==============================] - 2s 199us/step - loss: 76282.8260
Epoch 4/200
11934/11934 [==============================] - 2s 130us/step - loss: 64542.7553
Epoch 5/200
11934/11934 [==============================] - 1s 111us/step - loss: 63597.0720
Epoch 6/200
11934/11934 [==============================] - 2s 175us/step - loss: 63456.6514
Epoch 7/200
11934/11934 [==============================] - 2s 182us/step - loss: 63321.3331
Epoch 8/200
11934/11934 [==============================] - 2s 159us/step - loss: 63188.6389
Epoch 9/200
11934/11934 [==============================] - 2s 127us/step - loss: 63072.8953
Epoch 10/200
11934/11934 [==============================] - 1s 120us/step - loss: 62954.9515
Epoch 11/200
11934/11934 [==============================] - 1s 98us/step - lo

11934/11934 [==============================] - 1s 106us/step - loss: 60567.2488
Epoch 177/200
11934/11934 [==============================] - 1s 112us/step - loss: 60571.8355
Epoch 178/200
11934/11934 [==============================] - 1s 115us/step - loss: 60570.3026
Epoch 179/200
11934/11934 [==============================] - 1s 106us/step - loss: 60565.1278
Epoch 180/200
11934/11934 [==============================] - 1s 94us/step - loss: 60570.3762
Epoch 181/200
11934/11934 [==============================] - 1s 88us/step - loss: 60563.2447
Epoch 182/200
11934/11934 [==============================] - 1s 89us/step - loss: 60568.6183
Epoch 183/200
11934/11934 [==============================] - 1s 88us/step - loss: 60566.1892
Epoch 184/200
11934/11934 [==============================] - 1s 96us/step - loss: 60570.3501
Epoch 185/200
11934/11934 [==============================] - 1s 91us/step - loss: 60561.8525
Epoch 186/200
11934/11934 [==============================] - 1s 92us/step - loss

In [57]:
y_pred = model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  244.29129826663606
Std:  252.29854879882052
Mean:  373.96012064343165
Showing the first 10 differences between predicted and real.
445.996 874.0
430.00912 772.0
443.12375 591.0
297.7003 557.0
408.67172 582.0
519.3679 811.0
354.80054 292.0
400.2593 206.0
365.0933 954.0
382.17685 234.0


## Keras embedding + NN

### Generate input data 

In [58]:
X = []
for sample in w2v_corpus:
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Encode the words

In [59]:
encoder = LabelEncoder()
encoder.fit(np.array(X).flatten())

LabelEncoder()

In [60]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

### Split X in train and test

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_label, test_size=0.2, random_state=42)

In [62]:
INPUT_SHAPE = (FIXED_LENGTH, )
INPUT_SHAPE

(17,)

### Define the Keras embedded model 

In [63]:
def keras_embedding_model(encoder):
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH,), name='input')
    embedded_text = Embedding(input_dim=len(encoder.classes_),
                              output_dim=EMBEDDING_SIZE,
                              name='text_embedding')(input_layer)
    embedded_text = Flatten()(embedded_text)
    z = BatchNormalization()(embedded_text)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.7)(z)
    z = ReLU()(z)
    output = Dense(1, kernel_initializer='normal', activation='linear')(z)
    model = Model(inputs=[input_layer], outputs=[output])

    # Compile model
    model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')

    return model

In [64]:
keras_embedded_model = keras_embedding_model(encoder=encoder)

In [65]:
keras_embedded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 17)                0         
_________________________________________________________________
text_embedding (Embedding)   (None, 17, 50)            244400    
_________________________________________________________________
flatten_4 (Flatten)          (None, 850)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 850)               3400      
_________________________________________________________________
fc1 (Dense)                  (None, 32)                27232     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________

In [66]:
def schedule(epoch, lr):
    if epoch in [500]:
        return lr / 3.0
    return lr
scheduler = LearningRateScheduler(schedule=schedule)

In [67]:
y_series = pd.Series(y_label)
std_y, mean_y = y_series.describe()['std'], y_series.describe()['mean']

def rmse(y_true, y_pred):
    y_true = y_true * std_y + mean_y
    y_pred = y_pred * std_y + mean_y
    return K.sqrt(K.mean(K.square(y_true - y_pred), axis=0))

In [68]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [69]:
X_train.shape, y_train.shape

((11934, 17), (11934,))

In [70]:
history = keras_embedded_model.fit(X_train, y_train, epochs=EPOCHS,
                    callbacks=[scheduler, EarlyStopping(patience=10, monitor='rmse'),
                               ModelCheckpoint('models/embedding.h5', monitor='rmse')],
                    batch_size=BATCH_SIZE)

Epoch 1/200
11934/11934 [==============================] - 6s 502us/step - loss: 87702.8035


/home/skadi/.local/lib/python3.5/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `rmse` which is not available. Available metrics are: loss,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/200
11934/11934 [==============================] - 5s 409us/step - loss: 47386.0866
Epoch 3/200
11934/11934 [==============================] - 5s 403us/step - loss: 41887.5519
Epoch 4/200
11934/11934 [==============================] - 5s 423us/step - loss: 38785.5362
Epoch 5/200
11934/11934 [==============================] - 5s 427us/step - loss: 35579.1073
Epoch 6/200
11934/11934 [==============================] - 5s 448us/step - loss: 34168.8629
Epoch 7/200
11934/11934 [==============================] - 6s 490us/step - loss: 33300.6577
Epoch 8/200
11934/11934 [==============================] - 6s 480us/step - loss: 32622.6644
Epoch 9/200
11934/11934 [==============================] - 6s 513us/step - loss: 31754.9607
Epoch 10/200
11934/11934 [==============================] - 5s 450us/step - loss: 31051.5011
Epoch 11/200
11934/11934 [==============================] - 5s 443us/step - loss: 31434.4163
Epoch 12/200
11934/11934 [==============================] - 6s 476us/step - lo

11934/11934 [==============================] - 5s 447us/step - loss: 17477.7771
Epoch 178/200
11934/11934 [==============================] - 5s 438us/step - loss: 17166.1767
Epoch 179/200
11934/11934 [==============================] - 6s 471us/step - loss: 17748.2904
Epoch 180/200
11934/11934 [==============================] - 6s 467us/step - loss: 17297.1412
Epoch 181/200
11934/11934 [==============================] - 5s 442us/step - loss: 17487.1082
Epoch 182/200
11934/11934 [==============================] - 6s 501us/step - loss: 17205.9745
Epoch 183/200
11934/11934 [==============================] - 5s 460us/step - loss: 17453.1108
Epoch 184/200
11934/11934 [==============================] - 5s 459us/step - loss: 17178.7430
Epoch 185/200
11934/11934 [==============================] - 6s 498us/step - loss: 17582.0240
Epoch 186/200
11934/11934 [==============================] - 6s 492us/step - loss: 16966.9927
Epoch 187/200
11934/11934 [==============================] - 5s 448us/step

In [72]:
y_pred = keras_embedded_model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  237.7650888504333
Std:  252.29854879882052
Mean:  373.96012064343165
Showing the first 10 differences between predicted and real.
263.86 874.0
467.76233 772.0
1033.7655 591.0
192.15147 557.0
349.98022 582.0
863.8261 811.0
325.09338 292.0
367.44434 206.0
261.31796 954.0
192.54843 234.0


### Save model

In [ ]:
keras_embedded_model.save('models/predict_calories_model')

### Get the dataframe that has NaN values for our regression column (calories)

In [ ]:
predict_df = df[df[regr_column] != df[regr_column]]

### Get the input data we need to predict the needed values

In [ ]:
predict_data = []
for i in range(len(predict_df)):
    predict_data.append(get_small_dict(predict_df, keep_columns, i))
corpus = ['|'.join(
    d['tags'] + 
    [w.lower() for w in nltk.word_tokenize(d['title']) if w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
) for d in predict_data]

### Split input data using the tokenizer

In [ ]:
predict_corpus = []
for word_list in corpus:
    predict_corpus.append(tokenize(word_list))

### Make sure input data has a fixed shape

In [ ]:
X = []
for sample in predict_corpus:
    sample = [w for w in sample if w in encoder.classes_]
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Transform matrix-like input data using available encoder

In [ ]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

In [ ]:
X_encoded = np.array(X_encoded)
X_encoded.shape

### Predict missing data 

In [ ]:
y_pred = keras_embedded_model.predict(X_encoded)

y_pred = y_pred.flatten()

### Fill the dataframe with predicted values

In [ ]:
predict_df[regr_column] = y_pred

### Update initial dataframe with missing values

In [ ]:
update_regr_column = [np.nan for _ in range(len(df))]
for idx, value in zip(predict_df.index, predicted):
    update_regr_column[idx] = value

In [ ]:
df.update({regr_column: update_regr_column})

### Save updated dataframe

In [ ]:
df.to_csv('recipes_with_calories.csv', sep='|')